In [1]:
import reward_machines
import gymnasium 
from reward_machines.envs.grids.grid_world import *
from reward_machines.reward_machines.rm_environment import RewardMachineWrapper

In [26]:
#Make RMEnv (debugging in environment_test.ipynb)
rm_files = ["./reward_machines/envs/grids/reward_machines/rm2_one_feature.txt"]
map_file = "./reward_machines/envs/grids/maps/map1.txt"

test_env = ObstacleRMEnv(rm_files, map_file)

#Make RMEnv wrapper that generates counterfactual experience for each step
test_crm_env = RewardMachineWrapper(test_env, add_crm=True, gamma=1.0)

8
[['a' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' 'X' '.' '.' '.']
 ['.' '.' 'C' 'X' 'X' '.' '.' '.']
 ['.' '.' '.' 'X' '.' '.' '.' '.']
 ['.' 'H' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' 'B' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]


In [27]:
#Reset environment places agent back at starting location (0,0)
#and back at initial state in RM
test_crm_env.reset()

([Dict('features': Dict('a_loc': Box(0, 7, (2,), int64), 'c_loc': Box(0, 7, (2,), int64), 'b_loc': Box(0, 7, (2,), int64), 'h_loc': Box(0, 7, (2,), int64)), 'rm-state': Box(0, 1, (2,), uint8)),
  {'features': {'a_loc': array([0, 0]),
    'c_loc': array([2, 2]),
    'b_loc': array([6, 4]),
    'h_loc': [array([4, 1])]},
   'rm-state': array([1., 0.])}],
 {'distance_B': 10.0})

In [28]:
#Locations given by (row,col)
#(0,0) index in upper left corner rm_obs, rm_rew, done, info

#Agent actions that can be taken are:  
#0: right (+1 col)
#1: down  (+1 row)
#2: left  (-1 col)
#3: up    (-1 row)

#e.g.
#test_env.step(0) #takes a step right

#Taking a step out of bounds will not change the location
#e.g.
#test_env.step(3) #will not change location (going up sends out of bounds)

#Map is passed in map 


In [29]:
# #Good chunk of code for debugging to see what states and transitions exist in RM: 
# rm = test_crm_env.reward_machines[0]
# rm_states = rm.U
# for state in rm_states:
#     print(rm.delta_u[state]) #prints all transition possibilities (trigger propositions and resulting state) 

In [30]:
#Test what experience is generated after initialization (Success!)
#Reset environment
test_crm_env.reset()
#Take a step (doesn't allow movement)
rm_obs, rm_rew, done, info = test_crm_env.step(3)

#Here would expect experience to be generated for both states in RM 
#with different rewards (based on their differing reward functions)
print(info["crm-experience"])
print(len(info["crm-experience"]))

[([Dict('features': Dict('a_loc': Box(0, 7, (2,), int64), 'c_loc': Box(0, 7, (2,), int64), 'b_loc': Box(0, 7, (2,), int64), 'h_loc': Box(0, 7, (2,), int64)), 'rm-state': Box(0, 1, (2,), uint8)), {'features': {'a_loc': array([0, 0]), 'c_loc': array([2, 2]), 'b_loc': array([6, 4]), 'h_loc': [array([4, 1])]}, 'rm-state': array([1., 0.])}], 3, -1.0, [Dict('features': Dict('a_loc': Box(0, 7, (2,), int64), 'c_loc': Box(0, 7, (2,), int64), 'b_loc': Box(0, 7, (2,), int64), 'h_loc': Box(0, 7, (2,), int64)), 'rm-state': Box(0, 1, (2,), uint8)), {'features': {'a_loc': array([0, 0]), 'c_loc': array([2, 2]), 'b_loc': array([6, 4]), 'h_loc': [array([4, 1])]}, 'rm-state': array([1., 0.])}], False), ([Dict('features': Dict('a_loc': Box(0, 7, (2,), int64), 'c_loc': Box(0, 7, (2,), int64), 'b_loc': Box(0, 7, (2,), int64), 'h_loc': Box(0, 7, (2,), int64)), 'rm-state': Box(0, 1, (2,), uint8)), {'features': {'a_loc': array([0, 0]), 'c_loc': array([2, 2]), 'b_loc': array([6, 4]), 'h_loc': [array([4, 1])]}, 

In [31]:
#Test what experience is generated after reaching first target (Success!)

#resent env and travel to first target 
test_crm_env.reset() 
test_crm_env.step(0)
test_crm_env.step(0)
test_crm_env.step(0)
test_crm_env.step(0)
test_crm_env.step(0)
test_crm_env.step(1)
test_crm_env.step(1)
test_crm_env.step(1)
test_crm_env.step(1)
test_crm_env.step(1)
test_crm_env.step(1)
#last step to reach first target, expect to see experience calculated for first state 
#will both be negative 1 (because state 1 transition to state 2 has reward of -1, and state 2 has -1 as constant)
rm_obs, rm_rew, done, info = test_crm_env.step(2) 
print(info["crm-experience"])
print(len(info["crm-experience"]))

#one step after traveling to first target, don't expect experience to be calculated for first state
#(because no longer reachable given the sequence of actions performed thus far)
rm_obs, rm_rew, done, info = test_crm_env.step(2)
print(info["crm-experience"])
print(len(info["crm-experience"]))


[([Dict('features': Dict('a_loc': Box(0, 7, (2,), int64), 'c_loc': Box(0, 7, (2,), int64), 'b_loc': Box(0, 7, (2,), int64), 'h_loc': Box(0, 7, (2,), int64)), 'rm-state': Box(0, 1, (2,), uint8)), {'features': {'a_loc': array([6, 5]), 'c_loc': array([2, 2]), 'b_loc': array([6, 4]), 'h_loc': [array([4, 1])]}, 'rm-state': array([1., 0.])}], 2, -1, [Dict('features': Dict('a_loc': Box(0, 7, (2,), int64), 'c_loc': Box(0, 7, (2,), int64), 'b_loc': Box(0, 7, (2,), int64), 'h_loc': Box(0, 7, (2,), int64)), 'rm-state': Box(0, 1, (2,), uint8)), {'features': {'a_loc': array([6, 4]), 'c_loc': array([2, 2]), 'b_loc': array([6, 4]), 'h_loc': [array([4, 1])]}, 'rm-state': array([0., 1.])}], False), ([Dict('features': Dict('a_loc': Box(0, 7, (2,), int64), 'c_loc': Box(0, 7, (2,), int64), 'b_loc': Box(0, 7, (2,), int64), 'h_loc': Box(0, 7, (2,), int64)), 'rm-state': Box(0, 1, (2,), uint8)), {'features': {'a_loc': array([6, 5]), 'c_loc': array([2, 2]), 'b_loc': array([6, 4]), 'h_loc': [array([4, 1])]}, 'r

In [32]:
#Test traveling to hazard 

#Reset environment and travel to hazard
test_crm_env.reset()
test_crm_env.step(0)
test_crm_env.step(1)
test_crm_env.step(1)
test_crm_env.step(1)

#rm_obs, rm_new, done, info = test_crm_env.step(1)
# for entry in info["crm-experience"]:
#     rm_obs,action,rm_rew,rm_next_obs,done = entry
#     print(rm_next_obs)

#print(info["crm_experience"])

# print("current rm state is", test_crm_env.env.reward_machines[0].current_u_id)
# print("available state transitions are", test_crm_env.env.reward)

rm_obs, rm_new, done, info = test_crm_env.step(1)
print(done)

#Here would expect to learn over both states that transition to hazard with this action in this state 
print(info["crm-experience"])
print(len(info["crm-experience"]))

True
[([Dict('features': Dict('a_loc': Box(0, 7, (2,), int64), 'c_loc': Box(0, 7, (2,), int64), 'b_loc': Box(0, 7, (2,), int64), 'h_loc': Box(0, 7, (2,), int64)), 'rm-state': Box(0, 1, (2,), uint8)), {'features': {'a_loc': array([3, 1]), 'c_loc': array([2, 2]), 'b_loc': array([6, 4]), 'h_loc': [array([4, 1])]}, 'rm-state': array([1., 0.])}], 1, -100, [Dict('features': Dict('a_loc': Box(0, 7, (2,), int64), 'c_loc': Box(0, 7, (2,), int64), 'b_loc': Box(0, 7, (2,), int64), 'h_loc': Box(0, 7, (2,), int64)), 'rm-state': Box(0, 1, (2,), uint8)), {'features': {'a_loc': array([4, 1]), 'c_loc': array([2, 2]), 'b_loc': array([6, 4]), 'h_loc': [array([4, 1])]}, 'rm-state': array([0., 0.])}], True), ([Dict('features': Dict('a_loc': Box(0, 7, (2,), int64), 'c_loc': Box(0, 7, (2,), int64), 'b_loc': Box(0, 7, (2,), int64), 'h_loc': Box(0, 7, (2,), int64)), 'rm-state': Box(0, 1, (2,), uint8)), {'features': {'a_loc': array([3, 1]), 'c_loc': array([2, 2]), 'b_loc': array([6, 4]), 'h_loc': [array([4, 1])

In [33]:
#Test traveling to target 2 before target 1 (Success!)

#Reset environment 
test_crm_env.reset()
#Travel to second target
test_crm_env.step(0)
test_crm_env.step(0)
test_crm_env.step(1)
rm_obs, rm_new, done, info = test_crm_env.step(1)

#While first state will terminate negatively, should learn that second state terminates positively 
print(info["crm-experience"])

[([Dict('features': Dict('a_loc': Box(0, 7, (2,), int64), 'c_loc': Box(0, 7, (2,), int64), 'b_loc': Box(0, 7, (2,), int64), 'h_loc': Box(0, 7, (2,), int64)), 'rm-state': Box(0, 1, (2,), uint8)), {'features': {'a_loc': array([1, 2]), 'c_loc': array([2, 2]), 'b_loc': array([6, 4]), 'h_loc': [array([4, 1])]}, 'rm-state': array([1., 0.])}], 1, -100, [Dict('features': Dict('a_loc': Box(0, 7, (2,), int64), 'c_loc': Box(0, 7, (2,), int64), 'b_loc': Box(0, 7, (2,), int64), 'h_loc': Box(0, 7, (2,), int64)), 'rm-state': Box(0, 1, (2,), uint8)), {'features': {'a_loc': array([2, 2]), 'c_loc': array([2, 2]), 'b_loc': array([6, 4]), 'h_loc': [array([4, 1])]}, 'rm-state': array([0., 0.])}], True), ([Dict('features': Dict('a_loc': Box(0, 7, (2,), int64), 'c_loc': Box(0, 7, (2,), int64), 'b_loc': Box(0, 7, (2,), int64), 'h_loc': Box(0, 7, (2,), int64)), 'rm-state': Box(0, 1, (2,), uint8)), {'features': {'a_loc': array([1, 2]), 'c_loc': array([2, 2]), 'b_loc': array([6, 4]), 'h_loc': [array([4, 1])]}, '

In [2]:
#Make RMEnv (debugging in environment_test.ipynb)
rm_files = ["./reward_machines/envs/grids/reward_machines/rm0.txt"]
map_file = "./reward_machines/envs/grids/maps/map0.txt"
test_env = ObstacleRMEnv(rm_files, map_file)

#Make RMEnv wrapper that generates counterfactual experience for each step
test_crm_env = RewardMachineWrapper(test_env, add_crm=True, gamma=1.0)

5
[['a' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' 'B' '.' '.']]


In [3]:
test_crm_env.reset()

([Dict('features': Dict('a_loc': Box(0, 4, (2,), int64), 'b_loc': Box(0, 4, (2,), int64)), 'rm-state': Box(0, 1, (1,), uint8)),
  {'features': {'a_loc': array([0, 0]), 'b_loc': array([4, 2])},
   'rm-state': array([1.])}],
 {'distance_B': 6.0})

In [4]:
test_crm_env.step(0)

([Dict('features': Dict('a_loc': Box(0, 4, (2,), int64), 'b_loc': Box(0, 4, (2,), int64)), 'rm-state': Box(0, 1, (1,), uint8)),
  {'features': {'a_loc': array([0, 1]), 'b_loc': array([4, 2])},
   'rm-state': array([1.])}],
 -1,
 False,
 {'distance_B': 5.0,
  'crm-experience': [([Dict('features': Dict('a_loc': Box(0, 4, (2,), int64), 'b_loc': Box(0, 4, (2,), int64)), 'rm-state': Box(0, 1, (1,), uint8)),
     {'features': {'a_loc': array([0, 0]), 'b_loc': array([4, 2])},
      'rm-state': array([1.])}],
    0,
    -1,
    [Dict('features': Dict('a_loc': Box(0, 4, (2,), int64), 'b_loc': Box(0, 4, (2,), int64)), 'rm-state': Box(0, 1, (1,), uint8)),
     {'features': {'a_loc': array([0, 1]), 'b_loc': array([4, 2])},
      'rm-state': array([1.])}],
    False)]})

In [5]:
test_crm_env.step(0)

([Dict('features': Dict('a_loc': Box(0, 4, (2,), int64), 'b_loc': Box(0, 4, (2,), int64)), 'rm-state': Box(0, 1, (1,), uint8)),
  {'features': {'a_loc': array([0, 2]), 'b_loc': array([4, 2])},
   'rm-state': array([1.])}],
 -1,
 False,
 {'distance_B': 4.0,
  'crm-experience': [([Dict('features': Dict('a_loc': Box(0, 4, (2,), int64), 'b_loc': Box(0, 4, (2,), int64)), 'rm-state': Box(0, 1, (1,), uint8)),
     {'features': {'a_loc': array([0, 1]), 'b_loc': array([4, 2])},
      'rm-state': array([1.])}],
    0,
    -1,
    [Dict('features': Dict('a_loc': Box(0, 4, (2,), int64), 'b_loc': Box(0, 4, (2,), int64)), 'rm-state': Box(0, 1, (1,), uint8)),
     {'features': {'a_loc': array([0, 2]), 'b_loc': array([4, 2])},
      'rm-state': array([1.])}],
    False)]})